In [2]:
import json
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as psf
from pyspark import SparkContext
from pyspark.sql import SQLContext 

In [3]:
sc = SparkContext(appName="Wrangling-Project-Data")

In [4]:
sqlContext = SQLContext(sc)

In [5]:
path = 'police-department-calls-for-service.json'
df = sqlContext.read.json(path,multiLine=True)

In [6]:
df.printSchema()

root
 |-- address: string (nullable = true)
 |-- address_type: string (nullable = true)
 |-- agency_id: string (nullable = true)
 |-- call_date: string (nullable = true)
 |-- call_date_time: string (nullable = true)
 |-- call_time: string (nullable = true)
 |-- city: string (nullable = true)
 |-- common_location: string (nullable = true)
 |-- crime_id: string (nullable = true)
 |-- disposition: string (nullable = true)
 |-- offense_date: string (nullable = true)
 |-- original_crime_type_name: string (nullable = true)
 |-- report_date: string (nullable = true)
 |-- state: string (nullable = true)



In [7]:
## select original_crime_type_name and disposition
distinct_table = df.select("call_date_time","original_crime_type_name","disposition")
distinct_table.show()

+--------------------+------------------------+------------+
|      call_date_time|original_crime_type_name| disposition|
+--------------------+------------------------+------------+
|2018-12-31T23:57:...|            Traffic Stop|         ADM|
|2018-12-31T23:54:...|     Traf Violation Cite|         CIT|
|2018-12-31T23:49:...|            Passing Call|         HAN|
|2018-12-31T23:47:...|           Audible Alarm|         PAS|
|2018-12-31T23:46:...|            Traffic Stop|         CIT|
|2018-12-31T23:38:...|            Passing Call|          ND|
|2018-12-31T23:38:...|            Traffic Stop|         CIT|
|2018-12-31T23:36:...|            Traffic Stop|         HAN|
|2018-12-31T23:34:...|            Passing Call|Not recorded|
|2018-12-31T23:33:...|         Fight No Weapon|         GOA|
|2018-12-31T23:33:...|            Traffic Stop|         HAN|
|2018-12-31T23:33:...|            Passing Call|         HAN|
|2018-12-31T23:33:...|            Passing Call|         HAN|
|2018-12-31T23:30:...|  

In [23]:
## Select original crime type name and group by it
agg_df = distinct_table.select("*").withWatermark("call_date_time","3600 seconds").groupby("call_date_time","original_crime_type_name").count()
agg_df.show()

+------------------------+-----+
|original_crime_type_name|count|
+------------------------+-----+
|                 Footage|    7|
|                   Sword|    6|
|                 415/909|    1|
|            Physical/601|    1|
|                   915/x|    1|
|              Ne Tx 0123|    1|
|                 488/594|    2|
|                   Drink|    1|
|                 Dealing|    5|
|                 Erratic|    3|
|          594 W/ Bb Guns|    1|
|            Passing Call|35180|
|                     853|   30|
|          Person W/knife|  616|
|                     800|  218|
|                      Gz|   57|
|               Loud Bass|    3|
|                   2200e|    1|
|            Dropped Call|   14|
|              Music/ 909|    1|
+------------------------+-----+
only showing top 20 rows



In [24]:
radio_code_json_filepath = "radio_code.json"
radio_code_df = sqlContext.read.json(radio_code_json_filepath,multiLine=True)

# clean up your data so that the column names match on radio_code_df and agg_df
# we will want to join on the disposition code

# TODO rename disposition_code column to disposition
radio_code_df = radio_code_df.withColumnRenamed("disposition_code", "disposition")

In [25]:
## Count the number of original crime type
join_query = agg_df.join(agg_df.disposition == radio_code_df.disposition,how=="right")
join_query.show()

AttributeError: 'DataFrame' object has no attribute 'disposition'